In [3]:
import pulp

# 创建问题
model = pulp.LpProblem(name="nurse-scheduling-plan2", sense=pulp.LpMinimize)

# 决策变量：x[i,j]表示第i种休息组合，从第j个班次开始上班的护士数量
# i=1-5: 周六休息，周一到周五某天休息
# i=6-10: 周日休息，周一到周五某天休息
# j=1-5: 从第j个班次开始上班
x = {}
for i in range(1, 11):
    for j in range(1, 6):
        x[i, j] = pulp.LpVariable(f"x_{i}_{j}", lowBound=0, cat=pulp.LpInteger)

# 目标函数：最小化所需护士总数
model += pulp.lpSum(x[i, j] for i in range(1, 11) for j in range(1, 6))

# 辅助函数：获取某天某时间段工作的护士数量
def get_nurses_on_duty(day, time_slot):
    nurses = 0
    
    # 遍历所有休息组合和开始班次
    for i in range(1, 11):
        for j in range(1, 6):
            # 判断该护士在指定日期和时间段是否工作
            if is_nurse_on_duty(i, j, day, time_slot):
                nurses += x[i, j]
    
    return nurses

# 辅助函数：判断护士是否在指定日期和时间段工作
def is_nurse_on_duty(rest_combo, start_shift, day, time_slot):
    # 确定护士的休息日
    if rest_combo <= 5:  # 周六休息 + 周一到周五某天休息
        saturday_rest = True
        weekday_rest = rest_combo  # 1-5 对应周一到周五
    else:  # 周日休息 + 周一到周五某天休息
        saturday_rest = False
        weekday_rest = rest_combo - 5  # 6-10 对应周一到周五
    
    # 确定护士的工作日
    work_days = []
    for d in range(1, 8):  # 1-7 对应周一到周日
        if d == 6 and saturday_rest:  # 周六休息
            continue
        if d == 7 and not saturday_rest:  # 周日休息
            continue
        if d == weekday_rest:  # 周一到周五某天休息
            continue
        work_days.append(d)
    
    # 确定护士的工作班次
    shifts = []
    for s in range(5):  # 5个班次
        shift_day = (day - 1 + s) % 7 + 1  # 计算班次对应的日期
        if shift_day in work_days:
            shifts.append((s + 1) % 5 + 1)  # 1-5 对应5个班次
    
    # 判断护士是否在指定日期和时间段工作
    if day in work_days:
        shift_index = (day - 1 + start_shift - 1) % 5
        shift = shifts[shift_index] if shift_index < len(shifts) else None
        
        if shift is not None:
            # 判断班次是否覆盖指定时间段
            if time_slot == "6:00-10:00" and shift in [1, 2]:
                return True
            elif time_slot == "10:00-14:00" and shift in [2, 3]:
                return True
            elif time_slot == "14:00-18:00" and shift in [3, 4]:
                return True
            elif time_slot == "18:00-22:00" and shift in [4, 5]:
                return True
            elif time_slot == "22:00-6:00" and shift in [5, 1]:
                return True
    
    # 特殊处理夜班跨越午夜的情况
    # 如果护士在前一天工作，并且是夜班，那么她在休息日的凌晨（0:00-2:00）仍然在工作
    if time_slot == "22:00-6:00":
        prev_day = day - 1 if day > 1 else 7  # 前一天
        
        # 如果前一天是工作日
        if prev_day in work_days:
            prev_shift_index = (prev_day - 1 + start_shift - 1) % 5
            prev_shift = shifts[prev_shift_index] if prev_shift_index < len(shifts) else None
            
            # 如果前一天是第5班（18:00-2:00）
            if prev_shift == 5:
                return True
    
    return False

# 添加约束条件
time_slots = ["6:00-10:00", "10:00-14:00", "14:00-18:00", "18:00-22:00", "22:00-6:00"]
min_nurses = {"6:00-10:00": 18, "10:00-14:00": 20, "14:00-18:00": 19, "18:00-22:00": 17, "22:00-6:00": 12}

for day in range(1, 8):  # 1-7 对应周一到周日
    for time_slot in time_slots:
        model += get_nurses_on_duty(day, time_slot) >= min_nurses[time_slot]

# 求解问题
model.solve()

# 输出结果
print("方案2求解状态:", pulp.LpStatus[model.status])
total_nurses = 0
for i in range(1, 11):
    for j in range(1, 6):
        if pulp.value(x[i, j]) > 0:
            if i <= 5:
                print(f"x_{i}_{j} (周六休息，周{i}休息，从第{j}班开始): {pulp.value(x[i, j])}")
            else:
                print(f"x_{i}_{j} (周日休息，周{i-5}休息，从第{j}班开始): {pulp.value(x[i, j])}")
            total_nurses += pulp.value(x[i, j])
print(f"方案2需要的护士总数: {total_nurses}")

方案2求解状态: Optimal
x_1_3 (周六休息，周1休息，从第3班开始): 9.0
x_1_5 (周六休息，周1休息，从第5班开始): 5.0
x_3_2 (周六休息，周3休息，从第2班开始): 1.0
x_3_3 (周六休息，周3休息，从第3班开始): 1.0
x_3_5 (周六休息，周3休息，从第5班开始): 4.0
x_4_2 (周六休息，周4休息，从第2班开始): 7.0
x_4_3 (周六休息，周4休息，从第3班开始): 9.0
x_5_1 (周六休息，周5休息，从第1班开始): 11.0
x_6_1 (周日休息，周1休息，从第1班开始): 12.0
x_6_4 (周日休息，周1休息，从第4班开始): 1.0
x_6_5 (周日休息，周1休息，从第5班开始): 8.0
x_7_2 (周日休息，周2休息，从第2班开始): 12.0
x_7_4 (周日休息，周2休息，从第4班开始): 6.0
x_8_1 (周日休息，周3休息，从第1班开始): 8.0
x_8_4 (周日休息，周3休息，从第4班开始): 4.0
x_10_2 (周日休息，周5休息，从第2班开始): 7.0
方案2需要的护士总数: 105.0
